In [6]:
import nltk 
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
import os
os.environ["KERAS_BACKEND"] = "tensorflow"
import keras_nlp
import keras
import tensorflow as tf
from keras import layers
from keras.layers import TextVectorization
from dataclasses import dataclass
import pandas as pd
import numpy as np
import glob
import re
from pprint import pprint
from nltk.tokenize import WhitespaceTokenizer
import re 
import sys 

In [7]:
import importlib
import model 
import config
import dataset
from config import get_config, set_config_ratio
from dataset import get_dataset_class
from model import compile_model, get_optimizer
importlib.reload(model)
importlib.reload(dataset) 
importlib.reload(config)

<module 'config' from '/home/jovyan/config.py'>

In [8]:
config = get_config() 
dataset = get_dataset_class(config) 

In [4]:
mlm_ds = dataset.generate_dataset().shuffle(1000).batch(config.BATCH_SIZE)

2024-05-14 19:04:42.188863: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22400 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:89:00.0, compute capability: 8.6


starting generating masks...
starting generating mask for 0/50000 sentence...
finished generating mask for 1 sentence
starting generating mask for 1000/50000 sentence...
finished generating mask for 1 sentence
starting generating mask for 2000/50000 sentence...
finished generating mask for 1 sentence
starting generating mask for 3000/50000 sentence...
finished generating mask for 1 sentence
starting generating mask for 4000/50000 sentence...
finished generating mask for 1 sentence
starting generating mask for 5000/50000 sentence...
finished generating mask for 1 sentence
starting generating mask for 6000/50000 sentence...
finished generating mask for 1 sentence
starting generating mask for 7000/50000 sentence...
finished generating mask for 1 sentence
starting generating mask for 8000/50000 sentence...
finished generating mask for 1 sentence
starting generating mask for 9000/50000 sentence...
finished generating mask for 1 sentence
starting generating mask for 10000/50000 sentence...
f

In [13]:
class MaskedLanguageModel(keras.Model):
    def train_step(self, inputs):
        if len(inputs) == 3:
            features, labels, sample_weight = inputs
        else:
            features, labels = inputs
            sample_weight = None

        with tf.GradientTape() as tape:
            predictions = self(features, training=True)
            loss = loss_fn(labels, predictions, sample_weight=sample_weight)

        # Compute gradients
        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Compute our own metrics
        loss_tracker.update_state(loss, sample_weight=sample_weight)

        # Return a dict mapping metric names to current value
        return {"loss": loss_tracker.result()}

    @property
    def metrics(self):
        # We list our `Metric` objects here so that `reset_states()` can be
        # called automatically at the start of each epoch
        # or at the start of `evaluate()`.
        # If you don't implement this property, you have to call
        # `reset_states()` yourself at the time of your choosing.
        return [loss_tracker]

In [10]:
my_model = keras.models.load_model('saved_models/model_ratio_0.0', compile=False)
optim = get_optimizer()
my_model.compile(optimizer=optim, metrics=['accuracy'])

In [14]:
baseline_model = keras.models.load_model(
    "bert_mlm_imdb.keras", custom_objects={"MaskedLanguageModel": MaskedLanguageModel}
)

In [32]:
b_loss_fn = keras.losses.SparseCategoricalCrossentropy(reduction="none")
b_loss_tracker = keras.metrics.Mean(name="loss")
m_loss_fn = keras.losses.SparseCategoricalCrossentropy(reduction="none")
m_loss_tracker = keras.metrics.Mean(name="loss")

In [33]:
out = next(iter(mlm_ds.take(1)))
x, y, mask, padding_mask, sample_weights = out

In [34]:
base_out = baseline_model(x) 
my_out, attention = my_model((x, mask, padding_mask)) 

In [35]:
real_bout = base_out.numpy() 
real_mout = my_out.numpy() 

In [36]:
b_preds = np.argmax(real_bout[0], axis=-1)
m_preds = np.argmax(my_out[0], axis=-1) 

In [37]:
b_loss = b_loss_fn(y, base_out, sample_weight=sample_weights)
m_loss = m_loss_fn(y, my_out, sample_weight=sample_weights)

In [38]:
print(f"b loss: {b_loss}") 
print(f"m loss: {m_loss}")

b loss: [[0.         0.         0.         ... 2.9859133  0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.08079843 0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]
m loss: [[0.        0.        0.        ... 3.2821004 0.        0.       ]
 [0.        0.        0.        ... 0.        0.        0.       ]
 [0.4281662 0.        0.        ... 0.        0.        0.       ]
 ...
 [0.        0.        0.        ... 0.        0.        0.       ]
 [0.        0.        0.        ... 0.        0.        0.       ]
 [0.        0.        0.        ... 0.        0.        0.       ]]


In [39]:
b_loss_tracker.update_state(b_loss, sample_weight=sample_weights)
m_loss_tracker.update_state(m_loss, sample_weight=sample_weights) 
print(f"baseline loss: {b_loss_tracker.result()}")
print(f"my model loss: {m_loss_tracker.result()}")

baseline loss: 6.950923442840576
my model loss: 6.768109321594238


In [31]:
loss_tracker.update_state(m_loss, sample_weight=sample_weights)
print(f"loss: {loss_tracker.result()}")

loss: 7.006988048553467
